<a href="https://colab.research.google.com/github/martinpius/GANS/blob/main/Wasserstein_GAN_(WGAN)_IMPLEMENTATION_ON_CELEB_DATASET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/drive", force_remount = True)
try:
  COLAB = True
  import torch
  print(f">>>> You are on CoLaB. with torch version {torch.__version__}")
except Exception as e:
  print(f">>>> {type(e)} {e}\n>>>> please correct {type(e)} and reload your device")
  COLAB = False
def time_fmt(t: float = 123.187)->float:
  h = int(t / (60 * 60))
  m = int(t % (60 * 60) / 60)
  s = int(t % 60)
  return f"hrs: {h} min: {m:>02} sec: {s:>05.2f}"
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")
print(f">>>> testing the time formating function.........\n>>>> time elapsed\t{time_fmt()}")

Mounted at /content/drive
>>>> You are on CoLaB. with torch version 1.9.0+cu102
>>>> testing the time formating function.........
>>>> time elapsed	hrs: 0 min: 02 sec: 03.00


In [2]:
# In this notebook we are going to implement the Wasserstein GAN. WGAN is 
# very similar to DCGAN but in the training we employ the gradient cliping
# technique. WGAN has the advantages of reducing the impact of mode-collapse
# which is a ussual problems with simple and DCGANS. The architecture for the 
# discriminataor and the generator are extremely similar to DCGAN but we will
# not use sigmoid activation for the the discriminator. For demonstaration we 
# will use Flickr30k dataset.

In [3]:
import torch, torchvision
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from tqdm import tqdm
from PIL import Image
from tensorflow import summary
import pandas as pd
import numpy as np
import random, time, datetime, os
%load_ext tensorboard

In [4]:
# set the random seed for reproducability and the GPU to deterministic.
seed = 1234
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [5]:
# setup the tensorboard directories:
current_time = datetime.datetime.now().timestamp()
fake_path = "logs/tensorboard/wgan_fake/"+ str(current_time)
real_path = "logs/tensorboard/wgan_real/"+ str(current_time)
fake_writer = summary.create_file_writer(fake_path)
real_writer = summary.create_file_writer(real_path)


In [6]:
# The discriminator: This is very similar to the DCGAN with exception of sigmoid.
class Discriminator(nn.Module):
  def __init__(self, img_channels, d_features):
    super(Discriminator, self).__init__()
    self.discriminator = nn.Sequential(
        nn.Conv2d(img_channels, d_features, kernel_size = 4, stride = 2, padding = 1),
        nn.LeakyReLU(0.2),
        self.__dblock__(d_features, 2*d_features, 4, 2, 1),
        self.__dblock__(2*d_features, 4*d_features, 4, 2, 1),
        self.__dblock__(4*d_features, 8*d_features, 4, 2, 1),
        self.__dblock__(8*d_features, 1, kernel_size = 4, stride = 2, padding = 0),

    )
  
  def __dblock__(self, in_channels, out_channels, kernel_size, stride, padding):
    return nn.Sequential(
        nn.Conv2d(in_channels,
                  out_channels,
                  kernel_size,
                  stride,
                  padding,
                  bias = False),
        nn.BatchNorm2d(out_channels),
        nn.LeakyReLU(0.2))
  
  def forward(self, input_tensor):
    return self.discriminator(input_tensor)


In [7]:
# The generator class : similar to the DCGAN generator (use de-convolution to reconstruct the image)
class Generator(nn.Module):
  def __init__(self, z_dim, img_channels, g_features):
    super(Generator, self).__init__()
    self.generator = nn.Sequential(
        self.__deconv__(z_dim, 16* g_features, 4, 1, 0),
        self.__deconv__(16*g_features, 8*g_features, 4, 2, 1),
        self.__deconv__(8*g_features, 4*g_features, 4, 2, 1),
        self.__deconv__(4*g_features, 2*g_features, 4, 2, 1),
        nn.ConvTranspose2d(2*g_features, img_channels, kernel_size = 4, stride = 2, padding = 1),
        nn.Tanh())
    
  def __deconv__(self, in_channels, out_channels, kernel_size, stride, padding):
    return nn.Sequential(
        nn.ConvTranspose2d(in_channels,
                           out_channels,
                           kernel_size,
                           stride,
                           padding,
                           bias = False),
        nn.BatchNorm2d(out_channels),
        nn.ReLU())
  
  def forward(self, input_tensor):
    return self.generator(input_tensor)


In [8]:
# Initializing the weights to random normal with mean 0.0 and standard deviation of 0.02
def __initializer__(model):
  for m in model.modules():
    if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d, nn.BatchNorm2d, nn.InstanceNorm2d)):
      nn.init.normal_(m.weight.data, mean = 0.00, std = 0.02)

In [9]:
# Testing the network for the desired outputs:
def __test__():
  gen = Generator(100, 3, 64)
  disc = Discriminator(3, 64)
  noise = torch.randn(128, 100, 1, 1)
  rnd_img = torch.randn(128, 3, 64, 64)
  print(f">>>> discriminator's output shape: {disc(rnd_img).shape}\n>>>> generator's output shape: {gen(noise).shape}")

In [10]:
__test__()

>>>> discriminator's output shape: torch.Size([128, 1, 1, 1])
>>>> generator's output shape: torch.Size([128, 3, 64, 64])


In [11]:
# Hyperparameters 
learning_rate = 5e-5
batch_size = 128
d_features = 64
g_features = 64
img_channels = 3
z_dim = 100
wt_clip = 0.01
disc_iter = 5
img_size = 64
fixed_noise = torch.randn(batch_size, z_dim, 1, 1).to(device = device)
discriminator = Discriminator(img_channels, d_features).to(device = device)
generator = Generator(z_dim, img_channels, g_features).to(device = device)
__initializer__(generator)
__initializer__(discriminator)
transforms = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Resize((img_size, img_size)),
     transforms.Normalize([0.5 for _ in range(img_channels)], [0.5 for _ in range(img_channels)])]
)

In [12]:
# Getting the data and pre-process the data from google drive
class CustomData(Dataset):
  def __init__(self, root_dir, csv_file, transform = None):
    super(CustomData, self).__init__()
    self.root_dir = root_dir
    self.transform = transform
    self.dfm = pd.read_csv(csv_file, error_bad_lines = False)
  
  def __len__(self):
    return len(self.dfm)
  
  def __getitem__(self, idx):
    img_path = os.path.join(self.root_dir, self.dfm.iloc[idx, 0])
    image = Image.open(img_path)
    if self.transform:
      image = self.transform(image)
    return image


dataset = CustomData(root_dir = "/content/drive/MyDrive/flickr30k_images/flickr8k/images",
                     csv_file = "/content/drive/MyDrive/flickr30k_images/flickr8k/captions.txt",
                     transform = transforms)

loader = DataLoader(dataset = dataset, shuffle = True, batch_size = batch_size)
x_train_image = next(iter(loader))
print(f">>>> train image shape: {x_train_image.shape}")


>>>> train image shape: torch.Size([128, 3, 64, 64])


In [13]:
# Get the the optimizer objects for the training loop:
disc_optim = optim.RMSprop(params = discriminator.parameters(), lr = learning_rate)
gen_optim = optim.RMSprop(params = generator.parameters(), lr = learning_rate)


In [ ]:
# The training loop:
tic = time.time()
step = 0
EPOCHS = 10
for epoch in range(EPOCHS):
  print(f"\n>>>> training starts for epoch <{epoch + 1}>\n>>>> please wait while the model is training............................")
  for idx, real in enumerate(tqdm(loader)):
    real = real.to(device = device)
    noise = torch.randn(batch_size, z_dim, 1,1).to(device = device)
    fake = generator(noise)

    # training the discriminator with gradient cliping
    real_preds = discriminator(real).reshape(-1)
    fake_preds = discriminator(fake).reshape(-1)
    real_loss = torch.mean(real_preds)
    fake_loss = torch.mean(fake_preds)
    disc_loss = -(real_loss - fake_loss)
    discriminator.zero_grad()
    disc_loss.backward(retain_graph = True)
    disc_optim.step()

    #clipping the gradient for the discriminator net
    for m in discriminator.parameters():
      m.data.clamp_(-wt_clip, wt_clip)
    
    # training the generator as ussual:
    gen_preds = discriminator(fake).reshape(-1)
    gen_loss = -torch.mean(gen_preds)
    generator.zero_grad()
    gen_loss.backward()
    gen_optim.step()

    # print some losses on screen and to tensorboard:
    if idx % 100 == 0:
      print(f"\n>>>> end of epoch {epoch + 1}, generator loss: {gen_loss:.4f}, discriminator loss: {disc_loss:.4f}")

      fake = generator(fixed_noise)
      fake_img_grid = torchvision.utils.make_grid(fake[:64], normalize = True)
      real_img_grid = torchvision.utils.make_grid(real[:64], normalize = True)
      with fake_writer.as_default():
        summary.scalar("generator loss", gen_loss.cpu().detach().numpy(), step = step)
      
      with real_writer.as_default():
        summary.scalar("discriminator loss", disc_loss.cpu().detach().numpy(), step = step)
      step += 1

%tensorboard --logdir logs/tensorboard    



  0%|          | 0/317 [00:00<?, ?it/s]


>>>> training starts for epoch <1>
>>>> please wait while the model is training............................



  0%|          | 1/317 [00:30<2:38:46, 30.15s/it]


>>>> end of epoch 1, generator loss: -0.0001, discriminator loss: -0.0014



 32%|███▏      | 101/317 [26:03<26:27,  7.35s/it]

...


  0%|          | 1/317 [00:01<07:15,  1.38s/it]


>>>> end of epoch 10, generator loss: -0.0001, discriminator loss: -0.0036



 32%|███▏      | 101/317 [01:31<03:47,  1.05s/it]


>>>> end of epoch 10, generator loss: -0.0001, discriminator loss: -0.0037



 46%|████▌     | 145/317 [02:11<02:33,  1.12it/s]